# **Nav Canada Solution**

## **Util Functions**

In [1]:
!pip install flask flask-cors pyngrok
!pip install requests


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import re
import math
import requests
import re

def parse_coordinates(coord_str):
    """
    Convert a string like '50.43N/30.21E 51.00S/10.25W'
    into [[lat, lon], ...]
    """
    coords = coord_str.split()
    output = []

    pattern = r'([0-9]+(?:\.[0-9]+)?)([NS])/([0-9]+(?:\.[0-9]+)?)([EW])'

    for coord in coords:
        match = re.fullmatch(pattern, coord.strip())
        if not match:
            raise ValueError(f"Invalid coordinate format: {coord}")

        lat_value, lat_dir, lon_value, lon_dir = match.groups()

        lat = float(lat_value)
        lon = float(lon_value)

        if lat_dir == 'S':
            lat = -lat
        if lon_dir == 'W':
            lon = -lon

        output.append([lat, lon])

    return output


def parse_position_string_typed(s: str):
    """
    Returns:
    (position, radius, altitude, aradius, time)
    with numeric values converted to float.
    """
    parts = s.split('@')

    if len(parts) != 5:
        raise ValueError("Invalid format")

    position = parse_coordinates(parts[0])
    radius = float(parts[1])
    altitude = float(parts[2])
    aradius = float(parts[3])
    time = float(parts[4])

    return position, radius, altitude, aradius, time

def coordinateMetric(a,b):
  return math.abs(a[0]-b[0])+math.abs(a[1]-b[1])

def velocity2D(p0, p1, speed_knots):
    speed = speed_knots / 3600.0  # NM/s

    dx = p1[0] - p0[0]
    dy = p1[1] - p0[1]

    length = math.hypot(dx, dy)

    return (
        speed * dx / length,
        speed * dy / length
    )


def latlon_to_xy_nm(lat, lon):
    yInNM = (lat - SIM_ORIGIN_LAT) * 60
    avg_lat_rad = math.radians((lat + SIM_ORIGIN_LAT) / 2)
    xInNM = (lon - SIM_ORIGIN_LON) * 60 * math.cos(avg_lat_rad)
    return xInNM, yInNM


## **Global Variables**

In [3]:
#global variables

#Roughly central Canada
SIM_ORIGIN_LAT = 51.0
SIM_ORIGIN_LON = -114.0
MIN_VERTICAL_CLEARANCE=2000
MIN_HORIZONTAL_CLEARANCE=5

#key: ACID, example "WJA134"
#value: plane object
planeData=dict()

conflicts=[]

class Segment:
    def __init__(self, p0, v, t0, t1, altitude):
        self.position0 = p0
        self.velocity = v
        self.time0 = t0
        self.time1 = t1
        self.altitude = altitude

    def position(self, t):
        if not (self.time0 <= t <= self.time1):
            return None

        dt = t - self.time0
        return (
            self.position0[0] + self.velocity[0] * dt,
            self.position0[1] + self.velocity[1] * dt,
            self.altitude,
            t
        )



def buildSegment(wp0_latlon, wp1_latlon, t0, speed_knots, altitude):
    x0, y0 = latlon_to_xy_nm(*wp0_latlon)
    x1, y1 = latlon_to_xy_nm(*wp1_latlon)

    vx, vy = velocity2D((x0, y0), (x1, y1), speed_knots)

    distance = math.hypot(x1 - x0, y1 - y0)
    dt = distance / (speed_knots / 3600.0)

    return Segment(
        p0=(x0, y0),
        v=(vx, vy),
        t0=t0,
        t1=t0 + dt,
        altitude=altitude #feet
    )


class FlightLine:
    def __init__(self, segments):
        self.segments = segments

    def position(self, t):
        for seg in self.segments:
            pos = seg.position(t)
            if pos:
                return pos
        return None

    def __str__(self):
        lines = ["FlightLine:"]
        for i, seg in enumerate(self.segments, start=1):
            lines.append(
                f"  Segment {i}: "
                f"t=[{seg.time0:.1f}, {seg.time1:.1f}] s, "
                f"p0=({seg.position0[0]:.2f}, {seg.position0[1]:.2f}) nm, "
                f"v=({seg.velocity[0]:.2f}, {seg.velocity[1]:.2f}) nm/h, "
                f"alt={seg.altitude} ft"
            )
        return "\n".join(lines)



class Plane:
  def __init__(self, ACID, planeType, route, altitude, departureAirport, arrivalAirport, departureTime, aircraftSpeed, passengers, isCargo):
        self.ACID=ACID
        self.planeType=planeType
        self.route=route
        self.altitude=altitude
        self.departureAirport=departureAirport
        self.arrivalAirport=arrivalAirport
        self.departureTime=departureTime
        self.aircraftSpeed=aircraftSpeed
        self.passengers=passengers
        self.isCargo=isCargo




def calculateFlightLine(route, departureTime, speed_knots, altitude):
    segments = []
    t = departureTime

    for i in range(len(route) - 1):
        wp0 = route[i]
        wp1 = route[i + 1]

        #change units from latlon to nm
        x,y = latlon_to_xy_nm(wp0[0],wp0[1])
        x1,y1 = latlon_to_xy_nm(wp1[0],wp1[1])
        distance = math.sqrt((x1-x)**2 + (y1-y)**2)

        dt = distance / (speed_knots * 3600)

        seg = buildSegment(
            wp0,
            wp1,
            t,
            speed_knots=speed_knots,
            altitude=altitude
        )

        segments.append(seg)
        t += dt

    return FlightLine(segments)


#
class Conflict:
    def __init__(self, ACID1, ACID2, time0, time1, distance, posA, posB, segA_idx, segB_idx):
        self.ACID1 = ACID1
        self.ACID2 = ACID2
        self.time0 = time0
        self.time1 = time1
        self.distance = distance
        self.posA = posA
        self.posB = posB
        self.segA_idx = segA_idx
        self.segB_idx = segB_idx
    def __str__(self):
        return (
            f"Conflict {self.ACID1} ↔ {self.ACID2} | "
            f"t=[{self.time0:.1f}, {self.time1:.1f}] s | "
            f"dmin={self.distance:.2f} nm | "
            f"segA={self.segA_idx}, segB={self.segB_idx} | "
            f"posA=({self.posA[0]:.2f}, {self.posA[1]:.2f}) | "
            f"posB=({self.posB[0]:.2f}, {self.posB[1]:.2f})"
        )




## **Loading Data**

In [4]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

In [5]:
AIRPORT_COORDINATES = {
    "CYYZ": (43.68, -79.63),
    "CYVR": (49.19, -123.18),
    "CYUL": (45.47, -73.74),
    "CYYC": (51.11, -114.02),
    "CYOW": (45.32, -75.67),
    "CYWG": (49.91, -97.24),
    "CYHZ": (44.88, -63.51),
    "CYEG": (53.31, -113.58),
    "CYQB": (46.79, -71.39),
    "CYYJ": (48.65, -123.43),
    "CYYT": (47.62, -52.75),
    "CYXE": (52.17, -106.70),
}

In [6]:
def addDepartureCoordinates(plane):
  # ICAO → (latitude, longitude)
  departureAirport = plane.departureAirport
  departureCoordinates = AIRPORT_COORDINATES[departureAirport]
  plane.route.insert(0, departureCoordinates)

In [7]:
def addArrivalCoordinates(plane):
  # ICAO → (latitude, longitude)
  arrivalAirport = plane.arrivalAirport
  arrivalAirportCoordinates = AIRPORT_COORDINATES[arrivalAirport]
  plane.route.append(arrivalAirportCoordinates)

In [8]:
@app.route('/api/analyze', methods=['POST'])
def analyze_flights():
    data = request.get_json()
    raw_flights = data.get('flights', [])  # User's uploaded data
    
    # Your existing analysis code continues here...
    for flight in raw_flights:
        acid = flight["ACID"]
        planeData[acid] = Plane(...)
    # etc.

## **Detecting Conflicts**

In [9]:
  #detecting conflicts

def conflictBetweenSegments(ACID1, ACID2, segA, segB):
    # Vertical separation check
    if abs(segA.altitude - segB.altitude) >= 2000:
        return None

    t_start = max(segA.time0, segB.time0)
    t_end   = min(segA.time1, segB.time1)

    if t_start >= t_end:
        return None

    def pos(seg, t):
        dt = t - seg.time0
        return (
            seg.position0[0] + seg.velocity[0] * dt,
            seg.position0[1] + seg.velocity[1] * dt
        )

    # Relative position at t_start
    pA0 = pos(segA, t_start)
    pB0 = pos(segB, t_start)

    r0 = (pA0[0] - pB0[0], pA0[1] - pB0[1])
    vr = (segA.velocity[0] - segB.velocity[0],
          segA.velocity[1] - segB.velocity[1])

    a = vr[0]*vr[0] + vr[1]*vr[1]
    b = 2 * (r0[0]*vr[0] + r0[1]*vr[1])
    c = r0[0]*r0[0] + r0[1]*r0[1]

    D2 = MIN_HORIZONTAL_CLEARANCE * MIN_HORIZONTAL_CLEARANCE

    # Static relative motion
    if a == 0:
        if c > D2:
            return None

        return Conflict(
        ACID1=ACID1,
        ACID2=ACID2,
        time0=t_start,
        time1=t_end,
        distance=math.sqrt(c),
        posA=pA0,
        posB=pB0,
        segA_idx=None,
        segB_idx=None
    )


    # Solve quadratic: a t^2 + b t + (c - D2) = 0
    disc = b*b - 4*a*(c - D2)
    if disc < 0:
        return None

    sqrt_disc = math.sqrt(disc)

    tau1 = (-b - sqrt_disc) / (2*a)
    tau2 = (-b + sqrt_disc) / (2*a)

    # Convert to absolute times
    t_conflict_start = t_start + tau1
    t_conflict_end   = t_start + tau2

    # Clip to valid interval
    t0 = max(t_start, t_conflict_start)
    t1 = min(t_end,   t_conflict_end)

    if t0 >= t1:
        return None

    # Closest approach (for distance reporting)
    tau_cpa = -b / (2 * a)
    t_cpa = t_start + tau_cpa
    t_cpa = max(t0, min(t_cpa, t1))

    pA = pos(segA, t_cpa)
    pB = pos(segB, t_cpa)

    dx = pA[0] - pB[0]
    dy = pA[1] - pB[1]
    dist = math.hypot(dx, dy)

    return Conflict(
      ACID1,
      ACID2,
      t0,
      t1,
      dist,
      pA,
      pB,
      None,
      None
  )






def conflictBetweenFlightLines(ACID1, ACID2, wlA, wlB):
    earliest = None

    for i, segA in enumerate(wlA.segments):
        for j, segB in enumerate(wlB.segments):

            conflict = conflictBetweenSegments(ACID1, ACID2, segA, segB)

            if conflict:
                conflict.segA_idx = i
                conflict.segB_idx = j

                if earliest is None or conflict.time0 < earliest.time0:
                    earliest = conflict

    return earliest


def oneVsAllConflictBetweenFlights(ACID1, flightline1):
    local_conflicts = {}
    for plane in planeData.values():
        if plane.flightLine is flightline1:
            continue

        conflict = conflictBetweenFlightLines(ACID1, plane.ACID, flightline1, plane.flightLine)
        if conflict:
            local_conflicts.setdefault(conflict.time0, conflict)

    return local_conflicts


def allVsAllConflict():
    conflicts = {}
    planes = list(planeData.values())
    n = len(planes)
    for i in range(n):
        fl1 = planes[i].flightLine
        ACID1 = planes[i].ACID
        for j in range(i + 1, n):
            ACID2 = planes[j].ACID

            conflict = conflictBetweenFlightLines(ACID1, planes[j].ACID, fl1, planes[j].flightLine)
            if conflict:
                conflicts.setdefault(conflict.time0, conflict)

    return conflicts

conflicts = allVsAllConflict()
print(len(conflicts))

#for conflict in conflicts.values():
    #print(conflict)


0


## **Insights**

In [10]:
!apt-get -qq install libproj-dev proj-data proj-bin libgeos-dev
!pip install --quiet cartopy

conflict_list = list(conflicts.values())

#histogram of time of day of the collisions

from collections import Counter
import datetime
import matplotlib.pyplot as plt

hour_counts = Counter()

for c in conflict_list:
    hour = datetime.datetime.fromtimestamp(c.time0, tz=datetime.timezone.utc).hour
    hour_counts[hour] += 1


durations = [c.time1 - c.time0 for c in conflict_list]

mean_duration = sum(durations) / len(durations)
max_duration = max(durations)


# Make sure hours 0–23 are included, even if some have 0 counts
hours = list(range(24))
counts = [hour_counts.get(h, 0) for h in hours]

plt.figure(figsize=(10,5))
plt.plot(hours, counts, marker='o', color='dodgerblue', linewidth=2)
plt.xlabel("Hour of Day (UTC)")
plt.ylabel("Number of Conflicts")
plt.title("Conflicts by Hour of Day")
plt.xticks(hours)  # show every hour
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


#spatial center points

centers = [
    ((c.posA[0] + c.posB[0]) / 2,
     (c.posA[1] + c.posB[1]) / 2)
    for c in conflict_list
]


#aircraft involvement frequency
aircraft_counts = Counter()

for c in conflict_list:
    aircraft_counts[c.ACID1] += 1
    aircraft_counts[c.ACID2] += 1

# Get the top 10 planes by involvement
top_planes = aircraft_counts.most_common(10)
planes, counts = zip(*top_planes)  # unzip into two lists

plt.figure(figsize=(12,6))
plt.bar(planes, counts, color='orange')
plt.xlabel("Aircraft ID")
plt.ylabel("Number of Conflicts Involved")
plt.title("Top 10 Aircraft by Conflict Involvement")
plt.xticks(rotation=45)  # rotate labels if they are long
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


#making bins of conflict severity
severity_bins = {
    "<1NM": 0,
    "1–3NM": 0,
    "3–5NM": 0
}

for c in conflict_list:
    if c.distance < 1:
        severity_bins["<1NM"] += 1
    elif c.distance < 3:
        severity_bins["1–3NM"] += 1
    else:
        severity_bins["3–5NM"] += 1


# Data for the pie chart
labels = list(severity_bins.keys())
sizes = list(severity_bins.values())

plt.figure(figsize=(7,7))
plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',  # show percentage
    startangle=90,       # start from the top
    colors=['#ff9999','#66b3ff','#99ff99'],  # optional custom colors
    explode=(0.05, 0.05, 0.05)  # slightly separate the slices
)
plt.title("Conflict Severity Distribution")
plt.axis('equal')  # Equal aspect ratio ensures the pie is circular
plt.show()


#converting locations back to latitude and longitude to plot
def xy_nm_to_latlon(x, y):
    lat = SIM_ORIGIN_LAT + y / 60.0
    lon = SIM_ORIGIN_LON + x / (60.0 * math.cos(math.radians(SIM_ORIGIN_LON)))
    return lat, lon


import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.tight_layout()
plt.figure(figsize=(16,12))


# Convert centers to lat/lon
lats, lons = zip(*[xy_nm_to_latlon(x, y) for x, y in centers])

# Assign a color for each severity
severity_colors = {
    "<1NM": "red",
    "1–3NM": "orange",
    "3–5NM": "yellow"
}

# Create a list of colors corresponding to each conflict
colors = []
for c in conflict_list:
    if c.distance < 1:
        colors.append(severity_colors["<1NM"])
    elif c.distance < 3:
        colors.append(severity_colors["1–3NM"])
    else:
        colors.append(severity_colors["3–5NM"])

# Optionally, size points by duration (scaled)
#durations = [c.time1 - c.time0 for c in conflict_list]
#max_duration = max(durations)
sizes = [20 + 80 * (d / max_duration) for d in durations]  # min 20, max 100

# Determine map extent with a small margin
lat_margin = (max(lats) - min(lats)) * 0.1
lon_margin = (max(lons) - min(lons)) * 0.1
lat_min, lat_max = min(lats) - lat_margin, max(lats) + lat_margin
lon_min, lon_max = min(lons) - lon_margin, max(lons) + lon_margin

# Create map
plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

ax.set_aspect(1.0 / math.cos(math.radians((lat_min + lat_max)/2)))

# Plot conflicts
scatter = ax.scatter(lons, lats, c=colors, s=sizes, alpha=0.7, transform=ccrs.PlateCarree())

# Add legend for severity
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=key,
                          markerfacecolor=val, markersize=10)
                   for key, val in severity_colors.items()]
ax.legend(handles=legend_elements, title="Conflict Severity")

plt.title("Conflict Locations Map with Severity and Duration")
plt.show()




'apt-get' is not recognized as an internal or external command,
operable program or batch file.

[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ZeroDivisionError: division by zero

## **Hot Spot**

In [ ]:
import json, re, math
from collections import defaultdict
import numpy as np
import pandas as pd


# parse position

COORD_RE = re.compile(r"(\d+(?:\.\d+)?)([NS])\/(\d+(?:\.\d+)?)([EW])")

def parse_coord(token: str):
    """
    token example: "49.97N/110.935W"
    returns (lat, lon) float in signed degrees
    """
    m = COORD_RE.search(token.strip())
    if not m:
        return None
    lat, ns, lon, ew = m.groups()
    lat = float(lat) * (1 if ns == "N" else -1)
    lon = float(lon) * (1 if ew == "E" else -1)
    return lat, lon

def parse_route(route_str: str):
    """
    returns list of (lat, lon) in order from the route string
    """
    coords = []
    for match in COORD_RE.finditer(route_str):
        lat, ns, lon, ew = match.groups()
        lat = float(lat) * (1 if ns == "N" else -1)
        lon = float(lon) * (1 if ew == "E" else -1)
        coords.append((lat, lon))
    return coords

def floor_to_bin(ts, bin_seconds=900):
    # 900s = 15 minutes
    return int(ts // bin_seconds) * bin_seconds


# build "events" table

def build_events(flights, airport_latlon=None, bin_seconds=900):

    rows = []

    for f in flights:
        acid = f.get("ACID")
        dep_air = f.get("departure airport")
        arr_air = f.get("arrival airport")
        dep_t = int(f.get("departure time"))
        alt = float(f.get("altitude", np.nan))
        plane = f.get("Plane type") or f.get("plane type") or f.get("Plane Type")

        route = parse_route(f.get("route", ""))

        # crude duration estimate: assume 2 hours if unknown
        duration = int(f.get("duration_seconds", 2 * 3600))
        arr_t = dep_t + duration

        # Airport events
        rows.append({
            "entity_type": "airport",
            "entity_id": dep_air,
            "event_type": "dep",
            "t": dep_t,
            "bin": floor_to_bin(dep_t, bin_seconds),
            "ACID": acid,
            "altitude": alt,
            "plane": plane
        })
        rows.append({
            "entity_type": "airport",
            "entity_id": arr_air,
            "event_type": "arr",
            "t": arr_t,
            "bin": floor_to_bin(arr_t, bin_seconds),
            "ACID": acid,
            "altitude": alt,
            "plane": plane
        })

        # Waypoint events (spread uniformly between dep and arr)
        if len(route) > 0:
            # include each coordinate as its own waypoint id string for aggregation
            # (you can also snap to your "common waypoints" list if you want)
            for i, (lat, lon) in enumerate(route):
                if len(route) == 1:
                    t_i = dep_t
                else:
                    frac = i / (len(route) - 1)
                    t_i = int(dep_t + frac * duration)

                rows.append({
                    "entity_type": "waypoint",
                    "entity_id": f"{lat:.3f},{lon:.3f}",  # stable id
                    "event_type": "pass",
                    "t": t_i,
                    "bin": floor_to_bin(t_i, bin_seconds),
                    "ACID": acid,
                    "altitude": alt,
                    "plane": plane
                })

    return pd.DataFrame(rows)


# compute per-entity metrics per bin

def compute_metrics(events: pd.DataFrame):
    """
    Metrics per (entity_type, entity_id, bin):
      - traffic_density: count of unique flights
      - time_concentration: max flights in any 5-min subbin within the 15-min bin
      - altitude_overlap: fraction of flights in the busiest 2000-ft band (proxy)
    """
    if events.empty:
        return pd.DataFrame()

    # 5-minute subbin for concentration
    events = events.copy()
    events["subbin"] = (events["t"] // 300) * 300  # 300s = 5 min

    group_cols = ["entity_type", "entity_id", "bin"]

    # traffic density (unique flights)
    density = events.groupby(group_cols)["ACID"].nunique().rename("traffic_density")

    # time concentration (max unique flights in any 5-min subbin)
    conc = (
        events.groupby(group_cols + ["subbin"])["ACID"].nunique()
        .groupby(level=[0,1,2]).max()
        .rename("time_concentration")
    )

    # altitude overlap proxy:
    # For each group, bucket altitudes into 2000-ft bands, take max count / total
    def altitude_overlap_proxy(g):
        alts = g["altitude"].dropna().values
        if len(alts) == 0:
            return 0.0
        bands = (alts // 2000).astype(int)
        counts = pd.Series(bands).value_counts()
        return float(counts.max() / len(alts))

    alt_overlap = events.groupby(group_cols).apply(altitude_overlap_proxy).rename("altitude_overlap")

    metrics = pd.concat([density, conc, alt_overlap], axis=1).fillna(0).reset_index()
    return metrics

# -----------------------------
# Step C: normalize + pressure score + hotspot flag
# -----------------------------
def add_pressure_and_hotspots(metrics: pd.DataFrame,
                              weights=None,
                              hotspot_percentile=0.90,
                              multi_metric_rule=True):
    """
    Normalize each metric to [0,1] using percentiles and compute pressure.
    Hotspot rule:
      - pressure in top 10% (>= p90)
      - optionally: at least 2 or 3 metrics individually above p90
    """
    if metrics.empty:
        return metrics

    if weights is None:
        weights = {
            "traffic_density": 0.35,
            "time_concentration": 0.35,
            "altitude_overlap": 0.30
        }

    df = metrics.copy()

    metric_cols = list(weights.keys())

    # percentile normalization: value -> percentile rank in [0,1]
    for c in metric_cols:
        df[c + "_norm"] = df[c].rank(pct=True)

    # pressure score
    df["pressure"] = 0.0
    for c, w in weights.items():
        df["pressure"] += w * df[c + "_norm"]

    # hotspot threshold on pressure
    p_thr = df["pressure"].quantile(hotspot_percentile)
    df["hotspot_by_pressure"] = df["pressure"] >= p_thr

    # optional multi-metric rule: require several metrics above same percentile
    if multi_metric_rule:
        per_metric_flags = []
        for each in metric_cols:
            thr = df[each].quantile(hotspot_percentile)
            flag = df[each] >= thr
            per_metric_flags.append(flag.astype(int))
        df["num_metrics_high"] = np.sum(per_metric_flags, axis=0)
        # choose 2 or 3; 3 is stricter
        df["hotspot_by_multi"] = df["num_metrics_high"] >= 2
        df["hotspot"] = df["hotspot_by_pressure"] | df["hotspot_by_multi"]
    else:
        df["hotspot"] = df["hotspot_by_pressure"]

    return df

# -----------------------------
# Usage
# -----------------------------

events = build_events(raw_flights, bin_seconds=900)
metrics = compute_metrics(events)
scored = add_pressure_and_hotspots(metrics)
print(scored.sort_values("pressure", ascending=False).head(20))


## **Creating a working solution**

In [ ]:
import heapq
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, List, Set

# ----------------------------
# Helpers for altitude bands
# ----------------------------

ALTITUDE_CONSTRAINTS = {
    "regional": {"min": 22000, "max": 28000, "optimal": (24000, 26000)},
    "narrow_body": {"min": 28000, "max": 39000, "optimal": (33000, 37000)},
    "wide_body": {"min": 31000, "max": 43000, "optimal": (37000, 41000)},
    "cargo": {"min": 28000, "max": 41000, "optimal": (35000, 39000)},
}

AIRCRAFT_TYPE_MAP = {
    "Dash 8-400": "regional",
    "Embraer E195-E2": "regional",
    "Airbus A220-300": "regional",
    "Boeing 737-800": "narrow_body",
    "Boeing 737 MAX 8": "narrow_body",
    "Airbus A320": "narrow_body",
    "Airbus A321": "narrow_body",
    "Boeing 787-9": "wide_body",
    "Boeing 777-300ER": "wide_body",
    "Airbus A330": "wide_body",
    "Boeing 767-300F": "cargo",
    "Boeing 757-200F": "cargo",
    "Airbus A300-600F": "cargo",
}

MIN_VERTICAL_CLEARANCE = 2000
MIN_HORIZONTAL_CLEARANCE = 5

def aircraft_category(plane) -> str:
    return AIRCRAFT_TYPE_MAP.get(plane.planeType, ("cargo" if plane.isCargo else "narrow_body"))

def allowed_altitudes_for_plane(plane, step_ft=1000) -> List[int]:
    cat = aircraft_category(plane)
    lo, hi = ALTITUDE_CONSTRAINTS[cat]["min"], ALTITUDE_CONSTRAINTS[cat]["max"]
    opt_lo, opt_hi = ALTITUDE_CONSTRAINTS[cat]["optimal"]

    alts = list(range(lo, hi + 1, step_ft))

    # prioritize: inside optimal band first, then closest to current altitude
    alts.sort(key=lambda a: (0 if opt_lo <= a <= opt_hi else 1, abs(a - plane.altitude)))
    return alts

# ----------------------------
# Conflict key / priority
# ----------------------------

@dataclass(frozen=True)
class ConflictKey:
    a: str
    b: str
    t0: float  # entry time

def pair_key(a: str, b: str) -> Tuple[str, str]:
    return (a, b) if a < b else (b, a)

# ----------------------------
# Incremental conflict manager
# ----------------------------

class IncrementalConflictSet:
    """
    Maintains a set of CURRENT conflicts and a priority queue by earliest time0.
    Only recompute conflicts for planes that change.
    """
    def __init__(self, planeData: Dict[str, "Plane"]):
        self.planeData = planeData
        self.conflicts_by_pair: Dict[Tuple[str,str], "Conflict"] = {}  # store ONE (earliest) conflict per pair
        self.heap: List[Tuple[float, Tuple[str,str]]] = []            # (time0, pair)
        self.dirty_pairs: Set[Tuple[str,str]] = set()

    def build_all(self):
        acids = list(self.planeData.keys())
        n = len(acids)
        for i in range(n):
            a = acids[i]
            for j in range(i+1, n):
                b = acids[j]
                c = conflictBetweenFlightLines(a, b, self.planeData[a].flightLine, self.planeData[b].flightLine)
                if c:
                    pk = pair_key(a,b)
                    self.conflicts_by_pair[pk] = c
                    heapq.heappush(self.heap, (c.time0, pk))

    def _remove_pairs_involving(self, acid: str):
        # Just delete from dict; heap entries become stale and will be skipped lazily.
        to_delete = [pk for pk in self.conflicts_by_pair.keys() if acid in pk]
        for pk in to_delete:
            del self.conflicts_by_pair[pk]

    def recompute_for_plane(self, acid: str):
        """
        Remove all conflicts involving acid, then recompute acid vs all others.
        """
        self._remove_pairs_involving(acid)

        for other in self.planeData.keys():
            if other == acid:
                continue
            a, b = pair_key(acid, other)
            c = conflictBetweenFlightLines(a, b, self.planeData[a].flightLine, self.planeData[b].flightLine)
            if c:
                pk = (a,b)
                self.conflicts_by_pair[pk] = c
                heapq.heappush(self.heap, (c.time0, pk))

    def pop_earliest(self) -> Optional["Conflict"]:
        """
        Pop earliest valid conflict (skip stale heap items).
        """
        while self.heap:
            t0, pk = heapq.heappop(self.heap)
            c = self.conflicts_by_pair.get(pk)
            if c is None:
                continue
            # ensure heap entry matches current conflict time0 (might be stale)
            if abs(c.time0 - t0) > 1e-6:
                continue
            return c
        return None

    def has_conflicts(self) -> bool:
        return len(self.conflicts_by_pair) > 0

    def count(self) -> int:
        return len(self.conflicts_by_pair)

# ----------------------------
# Core solving logic
# ----------------------------

def plane_conflict_free(acid: str, planeData: Dict[str, "Plane"]) -> bool:
    """
    Check acid vs all others (no all-vs-all).
    """
    wlA = planeData[acid].flightLine
    for other in planeData.keys():
        if other == acid:
            continue
        a, b = pair_key(acid, other)
        c = conflictBetweenFlightLines(a, b, planeData[a].flightLine, planeData[b].flightLine)
        if c:
            return False
    return True

def try_fix_by_altitude(acid: str, planeData: Dict[str, "Plane"], step_ft=1000) -> bool:
    """
    Try to pick an altitude for this plane that produces NO conflicts vs anyone.
    """
    plane = planeData[acid]
    for alt in allowed_altitudes_for_plane(plane, step_ft=step_ft):
        if alt == plane.altitude:
            continue

        old_alt = plane.altitude
        plane.altitude = alt
        plane.flightLine = calculateFlightLine(plane.route, plane.departureTime, plane.aircraftSpeed, plane.altitude)

        if plane_conflict_free(acid, planeData):
            return True

        # rollback
        plane.altitude = old_alt
        plane.flightLine = calculateFlightLine(plane.route, plane.departureTime, plane.aircraftSpeed, plane.altitude)

    return False

def fix_by_delay_until_free(acid: str,
                            planeData: Dict[str, "Plane"],
                            delay_step_s: int = 5*60,
                            max_steps: int = 2000) -> bool:
    """
    Guaranteed termination lever: delay forward until conflict-free.
    """
    plane = planeData[acid]
    old_time = plane.departureTime

    for k in range(1, max_steps+1):
        plane.departureTime = old_time + k * delay_step_s
        plane.flightLine = calculateFlightLine(plane.route, plane.departureTime, plane.aircraftSpeed, plane.altitude)

        if plane_conflict_free(acid, planeData):
            return True

    # rollback if somehow not found
    plane.departureTime = old_time
    plane.flightLine = calculateFlightLine(plane.route, plane.departureTime, plane.aircraftSpeed, plane.altitude)
    return False

def choose_plane_to_modify(conflict) -> str:
    """
    Heuristic: modify the one with fewer passengers (or cargo first).
    """
    a = conflict.ACID1
    b = conflict.ACID2
    pa = planeData[a]
    pb = planeData[b]

    # cargo first, then fewer passengers
    score_a = (0 if pa.isCargo else 1, pa.passengers)
    score_b = (0 if pb.isCargo else 1, pb.passengers)
    return a if score_a <= score_b else b

def resolve_conflicts_incremental(planeData: Dict[str, "Plane"],
                                  max_iters: int = 20000,
                                  alt_step_ft: int = 1000,
                                  delay_step_s: int = 5*60) -> List[dict]:
    """
    Iteratively remove all conflicts using incremental updates.
    Altitude first (try to become globally conflict-free),
    otherwise delay forward until globally conflict-free.
    """
    # Build initial flightlines if missing
    for p in planeData.values():
        if not hasattr(p, "flightLine") or p.flightLine is None:
            p.flightLine = calculateFlightLine(p.route, p.departureTime, p.aircraftSpeed, p.altitude)

    cset = IncrementalConflictSet(planeData)
    cset.build_all()

    actions: List[dict] = []
    it = 0

    while cset.has_conflicts() and it < max_iters:
        it += 1
        c = cset.pop_earliest()
        if c is None:
            break

        target = choose_plane_to_modify(c)
        p = planeData[target]

        # 1) try altitude
        old_alt = p.altitude
        old_time = p.departureTime

        if try_fix_by_altitude(target, planeData, step_ft=alt_step_ft):
            actions.append({
                "iter": it,
                "acid": target,
                "action": "altitude",
                "from_alt": old_alt,
                "to_alt": planeData[target].altitude,
                "based_on_conflict": (c.ACID1, c.ACID2),
                "conflict_time0": c.time0
            })
            cset.recompute_for_plane(target)
            continue

        # 2) guaranteed fallback: delay
        if fix_by_delay_until_free(target, planeData, delay_step_s=delay_step_s):
            actions.append({
                "iter": it,
                "acid": target,
                "action": "delay",
                "from_time": old_time,
                "to_time": planeData[target].departureTime,
                "delay_min": (planeData[target].departureTime - old_time) / 60.0,
                "based_on_conflict": (c.ACID1, c.ACID2),
                "conflict_time0": c.time0
            })
            cset.recompute_for_plane(target)
            continue

        # If you ever land here, something is off (constraints too tight, bug, etc.)
        actions.append({
            "iter": it,
            "acid": target,
            "action": "failed",
            "based_on_conflict": (c.ACID1, c.ACID2)
        })
        break

    return actions

# ----------------------------
# Usage:
actions = resolve_conflicts_incremental(planeData)
print("actions:", len(actions))
# # After solver, verify:
final_conflicts = allVsAllConflict()
print("final conflicts:", len(final_conflicts))
def print_final_schedule(planeData):
    rows = []
    for acid, p in planeData.items():
        rows.append((acid, p.departureAirport, p.arrivalAirport, p.departureTime, p.altitude, p.aircraftSpeed, p.passengers, p.isCargo))
    rows.sort(key=lambda r: r[3])  # sort by departureTime

    print(f"{'ACID':8} {'DEP':5} {'ARR':5} {'DEP_TIME':12} {'ALT(ft)':8} {'SPD(kts)':8} {'PAX':5} {'CARGO':6}")
    for r in rows:
        acid, dep, arr, t, alt, spd, pax, cargo = r
        print(f"{acid:8} {dep:5} {arr:5} {int(t):12} {int(alt):8} {float(spd):8.1f} {int(pax):5} {str(cargo):6}")
actions = resolve_conflicts_incremental(planeData)
print_final_schedule(planeData)


In [ ]:
import base64
def image_to_base64(image_path):
    with open(image_path, 'rb') as f:
        return base64.b64encode(f.read()).decode()

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading


app = Flask(__name__)
CORS(app)

@app.route('/api/process', methods=['POST'])
def process_data():
    try:
        data = request.get_json()
        flights = data.get('data', [])
        
        # YOUR EXISTING ANALYSIS CODE HERE
        # Example response structure:
        response = {
            "summary": {
                "total_flights": len(flights),
                "total_passengers": sum(f.get('passengers', 0) for f in flights)
            },
            "tables": [
                {
                    "title": "Flight Summary",
                    "data": flights[:10]  # Your table data
                }
            ],
            "graphs": [
                # Your graphs as base64 images
            ]
        }
        
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run Flask in background
def run_flask():
    app.run(port=5000, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import json
import base64
import io

app = Flask(__name__)
CORS(app)

@app.route('/api/process', methods=['POST'])
def process_data():
    try:
        data = request.get_json()
        flights = data.get('data', [])
        
        # YOUR EXISTING ANALYSIS CODE HERE
        # Example response structure:
        response = {
            "summary": {
                "total_flights": len(flights),
                "total_passengers": sum(f.get('passengers', 0) for f in flights)
            },
            "tables": [
                {
                    "title": "Flight Summary",
                    "data": flights[:10]  # Your table data
                }
            ],
            "graphs": [
                # Your graphs as base64 images
            ]
        }
        
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run Flask in background
def run_flask():
    app.run(port=5000, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
